In [ ]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/spundsteif/spundsteif'
!pip3 install -e . -q
!pip3 install nbdev -q

#working directory: drive/MyDrive/spundsteif
%cd ..

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spundsteif/spundsteif
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/drive/MyDrive/spundsteif/spundsteif
/content/drive/MyDrive/spundsteif
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp pressures

# Library pressures
> This library gives the earth pressure distribution along the sheet piles from the total stress given by the sensors and the piezometer data.  
> Some functions aiming a graphical check of measurements are also available.  
--- 
**Inputs:**  
+ `df_messung`: dataframe of the measurements from sensors
+ `df_piezo.csv`: csv file of the groudwater level
+ `df_excavation.csv`: csv file of the stage of excavation

**Outputs:**  
+ `df_pressures` : total, effective stresses and waterpressure along the sheetpiles.


## Importation des bibliothèques

In [ ]:
#export
import pandas as pd
import numpy
import io
import csv
from datetime import datetime

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go 

import numpy as np

from spundsteif import sensors

## Test setup

### Test files

In [ ]:
PIEZO_FILE_NAME_TEST = './test/df_piezo_test.csv'
EXCAVATION_FILE_NAME_TEST = './test/df_excavation_stage_test.csv'

# Files needed to get df_measured_pressure
STRUCTURE_DATA = './test/1_structure_file.txt'

TESTS_FILES_NAMES = ['./test/1_data_test_01.ASC',
                     './test/1_data_test_02.txt',
                     './test/1_data_test_03.txt',
                     './test/1_data_test_04.ASC'
                     ]

TEST_HAND = './test/df_hand_test.csv'

### Measured pressure


In [ ]:
df_measured_pressure = sensors.get_data(list_files_names=TESTS_FILES_NAMES,
                                        structure_data=STRUCTURE_DATA, 
                                        file_name_df_hand=TEST_HAND)

df_measured_pressure = df_measured_pressure.loc[:,df_measured_pressure.columns.str.startswith('EDS')]

In [ ]:
df_measured_pressure.sample(10, random_state=0)

,EDS_A,EDS_B,EDS_C,EDS_D,EDS_E,EDS_F,EDS_G
2021-07-14 21:28:43.201370,-0.008392,0.443348,1.770932,-1.0,-0.013005,-6.332109,0.583440
2021-07-12 03:26:24.418810,-0.020542,-1.000000,-0.043438,-1.0,-0.013147,-6.065709,5.753562
2021-07-18 17:30:11.417100,0.011385,1.370416,2.557907,-1.0,0.001380,-6.130448,-3.851498
2021-07-15 19:29:03.205130,-0.003976,-1.000000,2.221444,-1.0,-0.013358,-6.334587,-1.914825
2021-07-08 19:25:08.005130,-0.015499,3.369134,-0.038738,-1.0,0.003474,-6.159601,-0.020800
2021-07-20 13:30:52.624630,0.017093,4.470687,2.422170,-1.0,0.000345,-6.302468,-4.388151
2021-08-18 00:00:00.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-19 13:30:29.620520,0.018375,4.841653,2.529231,-1.0,0.004594,-6.092389,-3.946954
2021-07-17 15:29:46.812660,0.010717,0.998889,2.490023,-1.0,0.006505,-6.300282,-3.930138
2021-07-08 14:25:04.204280,0.007363,1.635157,-0.019435,-1.0,0.025754,-6.187036,-0.024776


## Read files

### Read raw data

In [ ]:
#export
def construct_df_csv(file_name):
  df = pd.read_csv(file_name, delimiter=';')

  #remove empty lines and columns
  df = df.dropna(how='all')
  df = df.dropna(how='all', axis='columns')
  
  return df

In [ ]:
# example
df_piezo = construct_df_csv(PIEZO_FILE_NAME_TEST)
df_piezo

,Datum,Zeit,Tiefe/Piste [m],Temp /°C
0,06.07.2021,10:21:00,-1.10,14.4
1,07.07.2021,12:00:00,-1.05,17.3
2,14.07.2021,10:30:00,-0.82,14.5
3,21.07.2021,09:30:00,-2.23,13.1
4,28.07.2021,10:00:00,-1.01,12.3
5,05.08.2021,09:00:00,-0.92,13.0
6,11.08.2021,09:00:00,-1.33,14.8
7,18.08.2021,09:30:00,-1.30,18.0
8,25.08.2021,11:10:00,-1.52,13.6


In [ ]:
# example
df_excavation = construct_df_csv(EXCAVATION_FILE_NAME_TEST)
df_excavation

,Datum,Zeit,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
0,07.07.2021,07:30:00,1.1,1.1,1.0
1,21.07.2021,07:30:00,1.1,1.1,1.0
2,28.07.2021,08:30:00,1.8,1.1,0.5
3,18.08.2021,09:30:00,1.8,1.1,0.5
4,25.08.2021,10:30:00,1.8,1.1,0.5


### Clean data

In [ ]:
#export
def clean(df):
  
  # dtypes
  df['date'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')

  # remove 'Zeit' and 'Datum'
  df = df.drop(columns=['Datum', 'Zeit'])

  df = df.set_index('date')

  return df

In [ ]:
# example
df_piezo = clean(df_piezo)
df_piezo.dtypes

Tiefe/Piste [m]    float64
Temp /°C           float64
dtype: object

In [ ]:
# example
df_excavation= clean(df_excavation)
df_excavation.dtypes

Aushubstiefe/Verklebtebohlen [m]    float64
Aushubstiefe/Einzelbohlen [m]       float64
Tiefe/Aktivenseite [m]              float64
dtype: object

In [ ]:
# test
for df in df_piezo, df_excavation:
  assert pd.api.types.is_datetime64_any_dtype(df.index)
  assert 'Datum' not in df.columns
  assert 'Zeit' not in df.columns

assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Verklebtebohlen [m]'])
assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Einzelbohlen [m]'])

assert pd.api.types.is_float_dtype(df_piezo['Tiefe/Piste [m]'])
assert pd.api.types.is_float_dtype(df_piezo['Temp /°C'])

## Merge data

### Manual measures

All measures are depth in [m] from the top of the sheet piles

#### Merge piezo and excavation

In [ ]:
#export
def merge_piezo_excavation(df_piezo, df_excavation):
  df = df_piezo.join(df_excavation, how='outer')
  return df

In [ ]:
# example
df_manual = merge_piezo_excavation(df_piezo, df_excavation)
df_manual

,Tiefe/Piste [m],Temp /°C,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
date,,,,,
2021-07-06,-1.10,14.4,NaN,NaN,NaN
2021-07-07,-1.05,17.3,1.1,1.1,1.0
2021-07-14,-0.82,14.5,NaN,NaN,NaN
2021-07-21,-2.23,13.1,1.1,1.1,1.0
2021-07-28,-1.01,12.3,1.8,1.1,0.5
2021-08-05,-0.92,13.0,NaN,NaN,NaN
2021-08-11,-1.33,14.8,NaN,NaN,NaN
2021-08-18,-1.30,18.0,1.8,1.1,0.5
2021-08-25,-1.52,13.6,1.8,1.1,0.5


#### Clean manual measures

In [ ]:
#export
def clean_manual_df(df):
  df['GWT [m]'] = -1* (df['Tiefe/Piste [m]'] - 0.5)
  df = df.drop(columns=['Tiefe/Piste [m]'])
  return df

In [ ]:
# example
df_manual = clean_manual_df(df_manual)
df_manual

,Temp /°C,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,NaN,NaN,NaN,1.60
2021-07-07,17.3,1.1,1.1,1.0,1.55
2021-07-14,14.5,NaN,NaN,NaN,1.32
2021-07-21,13.1,1.1,1.1,1.0,2.73
2021-07-28,12.3,1.8,1.1,0.5,1.51
2021-08-05,13.0,NaN,NaN,NaN,1.42
2021-08-11,14.8,NaN,NaN,NaN,1.83
2021-08-18,18.0,1.8,1.1,0.5,1.80
2021-08-25,13.6,1.8,1.1,0.5,2.02


In [ ]:
# test
assert df_manual['GWT [m]']['2021-07-06'] == 1.6

#### Interpolate excavation

In [ ]:
#export
def interpolate_excavation(df):
  df = df.fillna(method='bfill')
  return df

In [ ]:
# example
df_manual = interpolate_excavation(df_manual)
df_manual

,Temp /°C,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,1.1,1.1,1.0,1.60
2021-07-07,17.3,1.1,1.1,1.0,1.55
2021-07-14,14.5,1.1,1.1,1.0,1.32
2021-07-21,13.1,1.1,1.1,1.0,2.73
2021-07-28,12.3,1.8,1.1,0.5,1.51
2021-08-05,13.0,1.8,1.1,0.5,1.42
2021-08-11,14.8,1.8,1.1,0.5,1.83
2021-08-18,18.0,1.8,1.1,0.5,1.80
2021-08-25,13.6,1.8,1.1,0.5,2.02


In [ ]:
# test
assert df_manual['Aushubstiefe/Verklebtebohlen [m]']['2021-07-06'] == 1.1
assert df_manual['Aushubstiefe/Einzelbohlen [m]']['2021-07-06'] == 1.1

#### Get manual dataframe

In [ ]:
#export
def get_manual_data(piezo_file_name, excavation_file_name):
  df_piezo = construct_df_csv(piezo_file_name)
  df_piezo = clean(df_piezo)

  df_excavation = construct_df_csv(excavation_file_name)
  df_excavation = clean(df_excavation)

  df = merge_piezo_excavation(df_piezo, df_excavation)
  df = clean_manual_df(df)
  df = interpolate_excavation(df)

  return df

In [ ]:
# example
df_manual = get_manual_data(PIEZO_FILE_NAME_TEST, EXCAVATION_FILE_NAME_TEST)
df_manual

,Temp /°C,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,1.1,1.1,1.0,1.60
2021-07-07,17.3,1.1,1.1,1.0,1.55
2021-07-14,14.5,1.1,1.1,1.0,1.32
2021-07-21,13.1,1.1,1.1,1.0,2.73
2021-07-28,12.3,1.8,1.1,0.5,1.51
2021-08-05,13.0,1.8,1.1,0.5,1.42
2021-08-11,14.8,1.8,1.1,0.5,1.83
2021-08-18,18.0,1.8,1.1,0.5,1.80
2021-08-25,13.6,1.8,1.1,0.5,2.02


### Merge manual and automatic measures

#### Merging

#### Clean

#### Interpolation

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 01_sensors.ipynb.
Converted 02_inclinometers.ipynb.
Converted 03_pressures.ipynb.
Converted index.ipynb.
